## Imports

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly.express as px
from bs4 import BeautifulSoup

### Carrega o arquivo 'remuneracao_servidores.csv' e cria um DataFrame a partir dele (rem_serv).

In [3]:
file_path = '../ds/remuneracao_servidores.csv'
rem_serv = pd.read_csv(file_path, sep=',', header=0, decimal='.')

try:       
    rem_serv.drop('Unnamed: 0', axis=1, inplace=True)
except:
    print("no unnamed column.")
        
rem_serv = rem_serv.sort_values(by=['ORGAO', 'DATA'])

orgaos = rem_serv["ORGAO"].drop_duplicates()
anos = rem_serv["ANO"].drop_duplicates().sort_values(ascending=False)
meses = rem_serv["MES"].drop_duplicates()

# Retira espaço inicial
rem_serv.rename(columns={' REMUNERACAO LEGAL DEVIDA(R$)' : 'REMUNERACAO LEGAL DEVIDA(R$)'}, inplace=True)

# Plotagens

## Órgãos que mais gastam
### Remuneração Total por Mês

In [15]:
df = rem_serv.loc[:, ['REMUNERACAO LEGAL TOTAL(R$)', 'DATA', 'ORGAO']].groupby(by=['ORGAO', 'DATA'], as_index=False).sum()

fig = px.line(df, title='Remuneração Legal Total de cada orgão ', x='DATA', y="REMUNERACAO LEGAL TOTAL(R$)", color='ORGAO')
# fig = px.scatter(df, title='Remuneração Legal Total de cada orgão ('+str(ano)+')', x='MES', y="REMUNERACAO LEGAL TOTAL(R$)", color='ORGAO', trendline='ols')
fig.show()

df_top = df[df['REMUNERACAO LEGAL TOTAL(R$)'] > 10000000]
fig = px.line(df_top, title='Órgãos com a soma de remunerações totais, por mês, maior que 10 milhões', x='DATA', y="REMUNERACAO LEGAL TOTAL(R$)", color='ORGAO')
fig = px.scatter(df_top, title='Órgãos com o total de remunerações maior que 10.000.000 (10 milhões)', x='DA', y="REMUNERACAO LEGAL TOTAL(R$)", color='ORGAO')
fig.show()

### Busca por órgão

In [33]:
orgao = 'PM-ATIVOS'
df = rem_serv.loc[(rem_serv.ORGAO == orgao), ['REMUNERACAO LEGAL TOTAL(R$)', 'DATA', 'ORGAO']].groupby(by=['DATA'], as_index=False).sum()

fig = px.line(df, title='Remuneração Legal Total - '+orgao, x='DATA', y='REMUNERACAO LEGAL TOTAL(R$)')
fig.show()

## Órgãos que tiveram aumento de um mês para o outro

In [13]:
pd.set_option('display.float_format', lambda x: '%.2f' % x) # Remove a notação científica

ano = 2018
mes = 8

df_mes_atual = rem_serv.loc[(rem_serv.MES == mes) & (rem_serv.ANO == ano),['ORGAO', 'ANO', 'MES', 'REMUNERACAO LEGAL TOTAL(R$)', 'DATA']].groupby(['ORGAO', 'DATA'], as_index=False)['REMUNERACAO LEGAL TOTAL(R$)'].sum()

if mes == 1:    
    df_mes_anterior = rem_serv.loc[(rem_serv.MES == 12) & (rem_serv.ANO == ano-1),:]
else:
    df_mes_anterior = rem_serv.loc[(rem_serv.MES == mes-1) & (rem_serv.ANO == ano),:]
    
df_mes_anterior = df_mes_anterior.groupby(['ORGAO', 'DATA'], as_index=False).sum()

rem_atual    = df_mes_atual[['ORGAO', 'REMUNERACAO LEGAL TOTAL(R$)', 'DATA']]
rem_anterior = df_mes_anterior[['ORGAO', 'REMUNERACAO LEGAL TOTAL(R$)', 'DATA']]

inner_join   = pd.merge(rem_anterior, rem_atual, how='inner', on='ORGAO', suffixes=(' ANTERIOR', ' ATUAL'))

orgs_aum     = pd.DataFrame()
orgs_aum['Remuneração Legal Total (Soma)'] = diff
orgs_aum['Órgão'] = rem_atual['ORGAO']

diff         = inner_join['REMUNERACAO LEGAL TOTAL(R$) ATUAL'] - inner_join['REMUNERACAO LEGAL TOTAL(R$) ANTERIOR']
diff         = diff.sort_values(ascending=False)

maior_diff   = diff.index.values.astype(int)[0]

plot         = px.line(inner_join, title='Variação da Remuneração Legal Total de cada órgão', x=[])

print("Maior diferença: \n", inner_join.iloc[maior_diff])
orgs_aum

Maior diferença: 
 ORGAO                                    PM-ATIVOS
REMUNERACAO LEGAL TOTAL(R$) ANTERIOR   67637115.70
DATA ANTERIOR                           2018-07-01
REMUNERACAO LEGAL TOTAL(R$) ATUAL      73984552.30
DATA ATUAL                              2018-08-01
Name: 30, dtype: object


,Remuneração Legal Total (Soma),Órgão
30,6347436.60,PM-ATIVOS
3,3761217.54,AMAZONPREV
32,2948860.17,POLICIA CIVIL
57,950113.87,SUSAM
41,928492.31,SEDUC
...,...,...
44,-86154.74,SEJEL
42,-137004.68,SEFAZ
29,-212030.53,PGE
38,-387286.38,SEAS


## Servidores duplicados no mesmo órgão

### Quantidade de servidores duplicados por órgão

In [27]:
df = rem_serv[['NOME', 'ANO', 'MES', 'ORGAO']]
df_bool = df.duplicated()
duplicados = df[df_bool]
duplicados_count = duplicados.groupby(['ORGAO'], as_index=False)['NOME'].count()
duplicados_count.rename(columns={'NOME': 'N_SERVIDORES'})

,ORGAO,N_SERVIDORES
0,ADAF,1
1,ALFREDO DA MATA,275
2,AMAZONPREV,143051
3,ARSAM,43
4,CASA CIVIL DO GOVERNO,103
5,CASA MILITAR,8
6,CETAM,301
7,CGE,36
8,CGL,23
9,DETRAN,5


### Busca de servidores duplicados (Órgão, Ano e Mês)

In [5]:
orgao = 'UEA'
ano = 2019
mes = 7

df = rem_serv.loc[(rem_serv.ORGAO == orgao) & (rem_serv.ANO == ano) & (rem_serv.MES == mes)]
duplicados = df[df.duplicated(['NOME'], keep=False)]
n_serv = int((duplicados['NOME'].count())/2)

print('Servidores duplicados:', n_serv)
duplicados
# rem_serv

Servidores duplicados: 2


,NOME,LOTACAO,CARGO,FUNCAO,VINCULO,REMUNERACAO LEGAL TOTAL(R$),DESC.TETO(R$),REMUNERACAO LEGAL DEVIDA(R$),DESCONTOS LEGAIS(R$),LIQUIDO DISPONIVEL(R$),ANO,MES,DATA,ORGAO
3347363,MARIA BETANIA LEAL DE OLIVEIRA,ESCOLA SUPERIOR DE TECNOLOGIA,P.AREAS EXAT.E TEC.4A.CL-NS.104.17,P AREAS EXATAS,ESTATUTARIO,6987.22,0.0,6987.22,1609.35,5377.87,2019,7,2019-07-01,UEA
3347364,MARIA BETANIA LEAL DE OLIVEIRA,ESCOLA SUP.DE TECNOLOGIA-EST,PROF.DOUTOR ADJ.C 40HS-PD.111.20,-,ESTATUTARIO,24932.86,0.0,24932.86,7322.29,17610.57,2019,7,2019-07-01,UEA
3347394,MARIA DO PERPETUO SOCORRO R LIMA,ESCOLA SUP.DE ARTES E TUR.ESAT,PROFESSOR C3 ED-ESP-III,-,ESTATUTARIO,6148.20,0.0,6148.20,1295.07,4853.13,2019,7,2019-07-01,UEA
3347395,MARIA DO PERPETUO SOCORRO R LIMA,ESCOLA SUP.DE ARTES E TUR.ESAT,PROFESSOR,-,ESTATUTARIO,6309.20,0.0,6309.20,1368.83,4940.37,2019,7,2019-07-01,UEA


## Servidores Duplicados em órgãos diferentes

In [28]:

df = rem_serv.loc[:,['NOME', 'ANO', 'MES', 'ORGAO']]

df = df.drop_duplicates(['NOME','ORGAO'])
df = df.groupby('NOME', as_index=False)[['ORGAO']].count()

df_bool = df[df['ORGAO'] > 1]
df_bool

nomes = df_bool['NOME']
nomes = rem_serv.merge(nomes, indicator=True, how='inner')['NOME']
nomes.drop_duplicates()

0                 AIRTON GONCALVES DOS SANTOS
33          ALEXANDRE HENRIQUE FREITAS ARAUJO
66                  AMILCAR DA SILVA FERREIRA
164         BARBARA BRANDAO FERREIRA SILVEIRA
198               DERLANI JOSE PEREIRA SERRAO
                         ...                 
498253          CIMARA BARROSO BRAGA DA SILVA
498269          DENIRA ISABEL GONCALVES SILVA
498294    MARA CRISTINA BATISTA DO NASCIMENTO
498320                   MARCEL PAREDES VALIN
498326                  THIRSO DEL CORSO NETO
Name: NOME, Length: 11042, dtype: object

### Busca de servidores duplicados (Ano e Nome)

In [10]:
ano = 2018
# serv_id = 36
nome = 'ADEJALMA CAMELO DA SILVA'
registros_serv = rem_serv.loc[(rem_serv.NOME == nome), :]
fig = px.line(registros_serv, title='Remuneração de '+nome, y='REMUNERACAO LEGAL TOTAL(R$)', x='DATA', color='ORGAO')
# fig = px.line(registros_serv, title='Remuneração de '+nomfes[serv_id]+', 2018', y='FUNCAO', x='ANO', color='ORGAO')
# teste = rem_serv.loc[(rem_serv.NOME == nomes[12]) & (rem_serv.ANO == ano), :]
# fig = px.line(teste, y='REMUNERACAO LEGAL TOTAL(R$)', x='MES', color='ORGAO')

fig.show()
registros_serv

,NOME,LOTACAO,CARGO,FUNCAO,VINCULO,REMUNERACAO LEGAL TOTAL(R$),DESC.TETO(R$),REMUNERACAO LEGAL DEVIDA(R$),DESCONTOS LEGAIS(R$),LIQUIDO DISPONIVEL(R$),ANO,MES,DATA,ORGAO
153990,ADEJALMA CAMELO DA SILVA,GABINETE,ASSESSOR II,ASSESSOR II,CARGO COMISSION.S/VINCULO,7717.79,0.0,7717.79,1703.28,6014.51,2018,5,2018-05-01,ADAF
154526,ADEJALMA CAMELO DA SILVA,GABINETE,ASSESSOR II,ASSESSOR II,CARGO COMISSION.S/VINCULO,4209.79,0.0,4209.79,670.28,3539.51,2018,6,2018-06-01,ADAF
153584,ADEJALMA CAMELO DA SILVA,GABINETE,ASSESSOR II,ASSESSOR II,CARGO COMISSION.S/VINCULO,4209.79,0.0,4209.79,670.28,3539.51,2018,7,2018-07-01,ADAF
155489,ADEJALMA CAMELO DA SILVA,GABINETE,ASSESSOR II,ASSESSOR II,CARGO COMISSION.S/VINCULO,4509.79,0.0,4509.79,670.28,3839.51,2018,8,2018-08-01,ADAF
152902,ADEJALMA CAMELO DA SILVA,GABINETE,ASSESSOR II,ASSESSOR II,CARGO COMISSION.S/VINCULO,4509.79,0.0,4509.79,670.28,3839.51,2018,9,2018-09-01,ADAF
155632,ADEJALMA CAMELO DA SILVA,GABINETE,ASSESSOR II,ASSESSOR II,CARGO COMISSION.S/VINCULO,4509.79,0.0,4509.79,670.28,3839.51,2018,10,2018-10-01,ADAF
155212,ADEJALMA CAMELO DA SILVA,GABINETE,ASSESSOR II,ASSESSOR II,CARGO COMISSION.S/VINCULO,4509.79,0.0,4509.79,670.28,3839.51,2018,11,2018-11-01,ADAF
154936,ADEJALMA CAMELO DA SILVA,GABINETE,ASSESSOR II,ASSESSOR II,CARGO COMISSION.S/VINCULO,4509.79,0.0,4509.79,670.28,3839.51,2018,12,2018-12-01,ADAF
156998,ADEJALMA CAMELO DA SILVA,GABINETE,ASSESSOR II,ASSESSOR II,CARGO COMISSION.S/VINCULO,7367.12,0.0,7367.12,3225.33,4141.79,2019,2,2019-02-01,ADAF
819475,ADEJALMA CAMELO DA SILVA,INATIVO DA POLICIA MILITAR,SOLDADO 1,-,ESTATUTARIO,2026.92,0.0,2026.92,0.00,2026.92,2017,5,2017-05-01,AMAZONPREV


## Admições e Demissões

In [12]:
mes = 5
df = rem_serv.loc[(rem_serv.ANO == 2019) & (rem_serv.ORGAO=='UEA') , ['NOME', 'ORGAO', 'ANO', 'MES']]

right_df = pd.merge(df.loc[(df.MES == mes-1), ['NOME', 'ORGAO']], df.loc[(df.MES == mes), ['NOME', 'ORGAO']], how='outer', indicator=True) 

demitidos = right_df.loc[(right_df._merge=='left_only'),['NOME']]
contratados = right_df.loc[(right_df._merge=='right_only'), ['NOME']]

print("Admissões:")
print(contratados)
print("\nDemissões:")
print(demitidos)

Admissões:
                                NOME
1673             ADRIANE NOBRE DINIZ
1674          EIDIE SOUZA DE QUEIROZ
1675  ERICK ANDERSON DO VALE TAVARES
1676  GUILHERME DE OLIVEIRA DOS REIS
1677          KELLY GOMES CAVALCANTE
1678    LEINA MARIA RODRIGUES ARRUDA
1679        VILCENCLEVER BRAGA ALVES

Demissões:
                                     NOME
328       DANIEL ÂNGELO OLIVEIRA DE ABREU
361         DENIRA ISABEL GONCALVES SILVA
387         DOROTHY IVILA DE MELO PEREIRA
420                   EIDIE DO VALE SOUZA
556        FRANCISCO LUIS HERNANDES ARIAS
712                 JEFERSON BARROS ALVES
800             JOSE ITALO SILVA DA ROCHA
853         JUSSARA MARIA PORDEUS E SILVA
865                  KATIA DE SOUZA PORTO
963                    LUCINA SOARES LIMA
970         LUIS BENIGNO CORRALES BARRIOS
1003                 MARCEL PAREDES VALIN
1280  PAULA ANGELA MARIA DE OLIVEIRA DIAS
1341             RAIMUNDO DA SILVA SEIXAS


### Busca remuneração individual

In [25]:
nome      = 'SEBASTIAO MONTEFUSCO CAVALCANTE'
busca     = rem_serv.loc[(rem_serv.NOME == nome)].sort_values('DATA')
fig_org   = px.line(busca, title='Remuneração Legal Total de '+nome+' por Órgão e Data', y='REMUNERACAO LEGAL TOTAL(R$)', x='DATA', color='ORGAO')
fig_cargo = px.line(busca, title='Remuneração Legal Total de '+nome+' por Cargo e Data', y='REMUNERACAO LEGAL TOTAL(R$)', x='DATA', color='CARGO')

fig_org.show()
fig_cargo.show()
busca

,NOME,LOTACAO,CARGO,FUNCAO,VINCULO,REMUNERACAO LEGAL TOTAL(R$),DESC.TETO(R$),REMUNERACAO LEGAL DEVIDA(R$),DESCONTOS LEGAIS(R$),LIQUIDO DISPONIVEL(R$),ANO,MES,DATA,ORGAO
842708,SEBASTIAO MONTEFUSCO CAVALCANTE,APOSENTADOS - SEFAZ,AUDITOR FISCAL DE TRIB.EST.1A.CL. V,-,ESTATUTARIO,89955.28,59484.17,30471.11,2134.93,28336.18,2017,5,2017-05-01,AMAZONPREV
842709,SEBASTIAO MONTEFUSCO CAVALCANTE,APOSENTADOS - SEDUC N-Z,PROFESSOR C4 ED-LPL-IV,-,ESTATUTARIO,1627.97,0.00,1627.97,0.00,1627.97,2017,5,2017-05-01,AMAZONPREV
680536,SEBASTIAO MONTEFUSCO CAVALCANTE,APOSENTADOS - SEFAZ,AUDITOR FISCAL DE TRIB.EST.1A.CL. V,-,ESTATUTARIO,89955.28,59484.17,30471.11,2134.93,28336.18,2017,6,2017-06-01,AMAZONPREV
680537,SEBASTIAO MONTEFUSCO CAVALCANTE,APOSENTADOS - SEDUC N-Z,PROFESSOR C4 ED-LPL-IV,-,ESTATUTARIO,1627.97,0.00,1627.97,0.00,1627.97,2017,6,2017-06-01,AMAZONPREV
1167698,SEBASTIAO MONTEFUSCO CAVALCANTE,APOSENTADOS - SEFAZ,AUDITOR FISCAL DE TRIB.EST.1A.CL. V,-,ESTATUTARIO,89955.28,59484.17,30471.11,2134.93,28336.18,2017,7,2017-07-01,AMAZONPREV
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654515,SEBASTIAO MONTEFUSCO CAVALCANTE,APOSENTADOS - SEDUC N-Z,PROFESSOR PF20.LPL-IV,-,ESTATUTARIO,2604.56,0.00,2604.56,0.00,2604.56,2019,11,2019-11-01,AMAZONPREV
321219,SEBASTIAO MONTEFUSCO CAVALCANTE,APOSENTADOS - SEFAZ,AUDITOR FISCAL DE TRIB.EST.1A.CL. V,-,ESTATUTARIO,109108.86,73646.64,35462.22,2616.17,32846.05,2019,12,2019-12-01,AMAZONPREV
321220,SEBASTIAO MONTEFUSCO CAVALCANTE,APOSENTADOS - SEDUC N-Z,PROFESSOR PF20.LPL-IV,-,ESTATUTARIO,2604.56,0.00,2604.56,0.00,2604.56,2019,12,2019-12-01,AMAZONPREV
1196773,SEBASTIAO MONTEFUSCO CAVALCANTE,APOSENTADOS - SEFAZ,AUDITOR FISCAL DE TRIB.EST.1A.CL. V,-,ESTATUTARIO,109108.86,73646.64,35462.22,2558.61,32903.61,2020,1,2020-01-01,AMAZONPREV


## Maiores aumentos

In [14]:
ano = 2018
mes = 7
df_mes_atual = rem_serv.loc[(rem_serv.MES == mes) & (rem_serv.ANO == ano),:].groupby(['ORGAO', 'ANO', 'MES', 'NOME'], as_index=False).head()

if mes == 1:    
    df_mes_anterior = rem_serv.loc[(rem_serv.MES == 12) & (rem_serv.ANO == ano-1),:]
else:
    df_mes_anterior = rem_serv.loc[(rem_serv.MES == mes-1) & (rem_serv.ANO == ano),:]

rem_atual = df_mes_atual[['NOME', 'REMUNERACAO LEGAL TOTAL(R$)', 'DATA']]
rem_anterior = df_mes_anterior[['NOME', 'REMUNERACAO LEGAL TOTAL(R$)', 'DATA']]
    
inner_join = pd.merge(rem_anterior, rem_atual, how='inner', on='NOME', suffixes=('_anterior', '_atual'))
diff = inner_join['REMUNERACAO LEGAL TOTAL(R$)_anterior'] - inner_join['REMUNERACAO LEGAL TOTAL(R$)_atual']
diff = diff.sort_values(ascending=False)
maior_diff = diff.index.values.astype(int)[0]
print("Maior diferença de salário:\n")
print(inner_join.iloc[maior_diff])
print("\nDiferenças de salário:")
diff

# df_final = pd.merge()
# diff = df_mes_atual['REMUNERACAO LEGAL TOTAL(R$)'] # - df_mes_anterior['REMUNERACAO LEGAL TOTAL(R$)']
# diff
# df_mes_atual
# df_mes_anterior
# df_mes_atual = rem_serv.loc[(),]
# df_mes_passado

Maior diferença de salário:

NOME                                    SEBASTIAO MONTEFUSCO CAVALCANTE
REMUNERACAO LEGAL TOTAL(R$)_anterior                            95989.5
DATA_anterior                                                2018-06-01
REMUNERACAO LEGAL TOTAL(R$)_atual                               1627.97
DATA_atual                                                   2018-07-01
Name: 42029, dtype: object

Diferenças de salário:


42029    94361.49
38958    84988.10
30776    82284.61
31206    81711.90
31205    80930.01
           ...   
31204   -80930.01
31201   -81711.90
30775   -82284.61
38959   -84933.50
42030   -94361.49
Length: 137850, dtype: float64

## Funcionarios que recebem acima do teto
### Remuneração Legal Total 
Que receberam mais que o teto na Remuneração Legal Total, ou seja, que tem descontos de teto maiores que 0.0

In [15]:
df = rem_serv.loc[(rem_serv['DESC.TETO(R$)'] > 0.0),:]
df.sort_values(by='DESC.TETO(R$)', ascending=False)

,NOME,LOTACAO,CARGO,FUNCAO,VINCULO,REMUNERACAO LEGAL TOTAL(R$),DESC.TETO(R$),REMUNERACAO LEGAL DEVIDA(R$),DESCONTOS LEGAIS(R$),LIQUIDO DISPONIVEL(R$),ANO,MES,DATA,ORGAO
847507,ATTYLA FILGUEIRA DA FONSECA,APOSENTADOS - SEFAZ,AUDITOR FISCAL DE TRIB.EST.1A.CL. V,-,ESTATUTARIO,122102.41,91631.30,30471.11,8149.05,22322.06,2018,12,2018-12-01,AMAZONPREV
354097,ATTYLA FILGUEIRA DA FONSECA,APOSENTADOS - SEFAZ,AUDITOR FISCAL DE TRIB.EST.1A.CL. V,-,ESTATUTARIO,123903.16,91631.30,32271.86,8149.05,24122.81,2018,10,2018-10-01,AMAZONPREV
1119141,ATTYLA FILGUEIRA DA FONSECA,APOSENTADOS - SEFAZ,AUDITOR FISCAL DE TRIB.EST.1A.CL. V,-,ESTATUTARIO,123903.16,91631.30,32271.86,8149.05,24122.81,2018,9,2018-09-01,AMAZONPREV
326364,ATTYLA FILGUEIRA DA FONSECA,APOSENTADOS - SEFAZ,AUDITOR FISCAL DE TRIB.EST.1A.CL. V,-,ESTATUTARIO,123903.16,91631.30,32271.86,8149.05,24122.81,2018,11,2018-11-01,AMAZONPREV
371095,MARIA SUMIE HARA SANTANA,APOSENTADOS - SEFAZ,AUDITOR FISCAL DE TRIB.EST.1A.CL. V,-,ESTATUTARIO,118479.18,88008.07,30471.11,2109.75,28361.36,2018,10,2018-10-01,AMAZONPREV
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569539,YOLANA MARIA GONCALVES KANEKO,APOSENTADOS SUSAM,MEDICO GRADUADO,-,ESTATUTARIO,7041.58,8.94,7032.64,1160.48,5872.16,2019,6,2019-06-01,AMAZONPREV
3279588,CARLOS ALBERTO FREITAS TUPINAMBA,GABINETE DO SECRETARIO,CORONEL,DIRETOR ADJUNTO IESP,ESTATUTARIO,6800.00,4.50,6795.50,760.49,6035.01,2019,10,2019-10-01,SSP
3274263,CARLOS ALBERTO FREITAS TUPINAMBA,GABINETE DO SECRETARIO,CORONEL,DIRETOR ADJUNTO IESP,ESTATUTARIO,6800.00,4.50,6795.50,812.63,5982.87,2019,12,2019-12-01,SSP
3285709,CARLOS ALBERTO FREITAS TUPINAMBA,GABINETE DO SECRETARIO,CORONEL,DIRETOR ADJUNTO IESP,ESTATUTARIO,6800.00,4.50,6795.50,812.63,5982.87,2019,11,2019-11-01,SSP


### Líquido Disponível
Que receberam mais que 35 mil de remuneração líquida.

In [16]:
df = rem_serv.loc[(rem_serv['LIQUIDO DISPONIVEL(R$)'] > 35000),:]
df = df.sort_values(by='LIQUIDO DISPONIVEL(R$)', ascending=False)
df.head(15)

,NOME,LOTACAO,CARGO,FUNCAO,VINCULO,REMUNERACAO LEGAL TOTAL(R$),DESC.TETO(R$),REMUNERACAO LEGAL DEVIDA(R$),DESCONTOS LEGAIS(R$),LIQUIDO DISPONIVEL(R$),ANO,MES,DATA,ORGAO
1213056,AMAZONINO ARMANDO MENDES,CASA CIVIL,GOVERNADOR DE ESTADO,-,CARGO COMISSION.S/VINCULO,545748.93,0.00,545748.93,0.00,545748.93,2018,12,2018-12-01,CASA CIVIL DO GOVERNO
775932,JOZIEL SILVA LOUREIRO,APOSENTADOS - POLICIA CIVIL,DELEGADO DE POLICIA 4A.CL.PC-DEL-IV,-,ESTATUTARIO,269385.16,0.00,269385.16,54364.27,215020.89,2019,3,2019-03-01,AMAZONPREV
1213091,ANTONIO SANTINO DE SOUZA,CASA CIVIL,A.TEC-I,-,ESTATUTARIO,246765.07,0.00,246765.07,84616.05,162149.02,2018,12,2018-12-01,CASA CIVIL DO GOVERNO
1074685,MARIA DA FE LIRA DA ROCHA,APOSENTADOS - SEFAZ,TEC.DE ARREC.DE TRIB.EST.1A.CL.II,-,ESTATUTARIO,157075.11,1443.03,155632.08,2616.17,153015.91,2019,9,2019-09-01,AMAZONPREV
764854,ALDEMIR DOCE DA FONSECA,APOSENTADOS - SEFAZ,AUDITOR FISCAL DE TRIB.EST.1A.CL. V,-,ESTATUTARIO,220881.49,0.00,220881.49,73687.80,147193.69,2019,3,2019-03-01,AMAZONPREV
728234,MARIA LUIZA CARVALHO DE MENEZES,APOSENTADOS - SEFAZ,TEC.DA FAZENDA ESTADUAL 1A.CL.V,-,ESTATUTARIO,146423.92,4575.04,141848.88,33337.59,108511.29,2019,7,2019-07-01,AMAZONPREV
8058,PAULO JOSE GOMES DE CARVALHO,GAB DO SUBPROCURADOR GERAL,PROCURADOR DO ESTADO 1A CLASSE,PROCURADOR-GERAL,ESTATUTARIO,138033.78,15608.05,122425.73,14862.23,107563.50,2018,1,2018-01-01,PGE
924523,ALDEMIR DOCE DA FONSECA,APOSENTADOS - SEFAZ,AUDITOR FISCAL DE TRIB.EST.1A.CL. V,-,ESTATUTARIO,159154.95,0.00,159154.95,51959.98,107194.97,2018,5,2018-05-01,AMAZONPREV
1674453,LIETE NASCIMENTO DOS SANTOS,PENSIONISTAS - APOSENT. SEFAZ,PENSIONISTA,79/99,ESTATUTARIO,194533.77,34373.48,160160.29,54973.65,105186.64,2018,4,2018-04-01,PENSIONISTAS
1641942,MARIA JANDIRA MELO DE OLIVEIRA,PENSIONISTAS - APOSENT. SEFAZ,PENSIONISTA,477/2010,ESTATUTARIO,149868.97,0.00,149868.97,51322.80,98546.17,2018,5,2018-05-01,PENSIONISTAS
